**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [70]:
import gzip
import requests
from bs4 import BeautifulSoup

In [93]:
# url='http://lib.ru/NEWPROZA/LOBAS/taxisty.txt'
# text = BeautifulSoup(requests.get(url).text).get_text()
# with gzip.open('taxisty.txt.gz','wt') as f: f.write(text)

# # with gzip.open('taxisty.txt.gz','rt') as f: text = f.read()

# text = text[1030:-7261].strip() # выкидываем заголовок и хвост страницы 
# print(f'символов:{len(text)}\n---------------\n'%())
# print(text[:343])

In [87]:
url='http://az.lib.ru/d/dostoewskij_f_m/text_0080.shtml'
text = BeautifulSoup(requests.get(url).text).get_text()
with gzip.open('dostoewskij.txt.gz','wt') as f: f.write(text)

# with gzip.open('dostoewskij.txt.gz','rt') as f: text = f.read()

text = text[2876:-664184].strip() # выкидываем заголовок и хвост страницы 
print(f'символов:{len(text)}\n---------------\n'%())
print(text[:355])

In [88]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [89]:
from tqdm.auto import tqdm
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

sentences = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in tqdm(nltk_sentence_split(text)) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(sentences)))
display( sample(sentences,1) )

  0%|          | 0/6685 [00:00<?, ?it/s]

предложений: 6685



[['Гелию',
  'и',
  'его',
  'сокамернику',
  'вздумалось',
  'устроить',
  'в',
  'тюрьме',
  'на',
  'Новый',
  'год',
  '--',
  'елочку',
  '...',
  'Конечно',
  'же',
  ',',
  'заключенный',
  'играл',
  'с',
  'капитаном',
  ',',
  'как',
  'кошка',
  'с',
  'мышкой',
  ',',
  'но',
  'дураковатый',
  'капитан',
  'клюнул',
  'на',
  'удочку',
  'и',
  'не',
  'только',
  'разрешил',
  'неслыханное',
  'в',
  'следственном',
  'изоляторе',
  'КГБ',
  'баловство',
  ',',
  'а',
  'собственноручно',
  'принес',
  'заключенным',
  'две',
  'или',
  'три',
  'пахнущие',
  'смолой',
  'и',
  'морозом',
  'хвойные',
  'ветки',
  '.']]

In [90]:
sentences = sentences[:128] # ограничиваем датасет для ускорения процеса 

In [119]:
from nltk.lm.preprocessing import padded_everygram_pipeline 

ngram_len = 3

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(ngram_len, sentences)

# собираем модель

# from nltk.lm import MLE as LangModel 
from nltk.lm import Laplace as LangModel

model = LangModel(ngram_len) 
model.fit(train, vocab)

display(len(model.vocab))

1654

In [120]:
# генерируем продолжения
for sentence in sample(sentences,30): # выбираем рандомно 10 предложений
    if len(sentence)<10: continue
    # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    # генерируем возможные продолжения
    result = model.generate(3, text_seed=sentence_) 
    print(  ' '.join(sentence_)  + ' ... ' + str( result ) + '\n' )

Завидные деньги , однако , никому не даются ... ['даром', '.', '</s>']

Любому из моих друзей ты можешь оставить на часок в центре города свой `` Мерседес '' без всякого риска , что машину уволокут на штрафную площадку ; захочешь .понюхать '' или `` пустить по вене '' , поставить на бейсбольную команду или же пожелаешь добавить в свою жизнь капельку нежности -- они всегда позаботятся , чтобы ты получил именно то , к чему нынче предрасположен ... А кроме швейцаров я познакомлю тебя с кинозвездой-собакой и с нищим таксистом , который разбогател в течение считанных дней : я покажу тебе , читатель , тот особый Нью-Йорк , каким его видят водители желтых кэбов , а если тебе любопытно -- научу , как ... ['я', ',', 'эмигрантов']

Как ни старался Ангел даже не глядеть в ту сторону , более часу вытерпеть он не мог и посылал за порцией ... ['кого-нибудь', 'из', 'пользовавшихся']

Ангел стоял на своем посту , терзаемый головоломной ... ['мыслью', '.', '</s>']

Они же горло готовы перегрызть за сво

In [121]:
# from nltk.util import bigrams
from nltk.util import ngrams

# оцениваем насколько хорошо модель предсказывает слова из датасета
text_ngrams = [ ng for s in sentences for ng in ngrams(s,3) ]

print( 'perplexity:', model.perplexity( text_ngrams ) )

perplexity: 816.8923634600944


In [122]:
display( text_ngrams[:3] )

[('Вы', 'прилетели', 'в'),
 ('прилетели', 'в', 'Нью-Йорк'),
 ('в', 'Нью-Йорк', 'и')]